In [17]:
import pandas as pd
import sasoptpy as so 
#Sertelp goes to FPLReview.com to get the .csv data

In [18]:
df = pd.read_csv('../data/fpl_data.csv')

In [19]:
df.head()

,Unnamed: 0,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,value
0,0,Xhaka,Arsenal,3,2.6,49,2113,845792,19.2,94,Midfielder,19.2
1,1,Elneny,Arsenal,3,0.6,41,111,198337,1.5,6,Midfielder,1.5
2,2,Holding,Arsenal,2,0.1,42,17,7171,1.7,7,Defender,1.7
3,3,Partey,Arsenal,3,0.4,47,1681,101282,14.3,67,Midfielder,14.3
4,4,Ødegaard,Arsenal,3,37.3,70,2131,5467369,20.6,144,Midfielder,20.6


In [20]:
gk_data = df[df['position'] == 'Goalkeeper'].copy().reset_index()

In [21]:
gk_data.head()

,index,Unnamed: 0,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,value
0,11,12,Ramsdale,Arsenal,1,12.1,49,2340,1469716,21.0,103,Goalkeeper,21.0
1,24,35,Martínez Romero,Aston Villa,1,4.4,49,2014,579222,15.9,78,Goalkeeper,15.9
2,27,39,Olsen,Aston Villa,1,1.6,39,235,114676,2.3,9,Goalkeeper,2.3
3,60,87,Travers,Bournemouth,1,0.4,42,945,63760,6.0,25,Goalkeeper,6.0
4,68,98,Murara Neto,Bournemouth,1,0.2,45,1305,60431,10.4,47,Goalkeeper,10.4


In [22]:
model = so.Model(name='gk_model')

NOTE: Initialized model gk_model.


In [23]:
gk_data.set_index('index', inplace = True)

In [24]:
gk_data.head()

,Unnamed: 0,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points,position,value
index,,,,,,,,,,,,
11,12,Ramsdale,Arsenal,1,12.1,49,2340,1469716,21.0,103,Goalkeeper,21.0
24,35,Martínez Romero,Aston Villa,1,4.4,49,2014,579222,15.9,78,Goalkeeper,15.9
27,39,Olsen,Aston Villa,1,1.6,39,235,114676,2.3,9,Goalkeeper,2.3
60,87,Travers,Bournemouth,1,0.4,42,945,63760,6.0,25,Goalkeeper,6.0
68,98,Murara Neto,Bournemouth,1,0.2,45,1305,60431,10.4,47,Goalkeeper,10.4


In [25]:
players = gk_data.index.tolist()

In [26]:
players

[11,
 24,
 27,
 60,
 68,
 78,
 101,
 108,
 129,
 139,
 156,
 180,
 185,
 206,
 217,
 237,
 271,
 294,
 321,
 337,
 364,
 382,
 399,
 404,
 416,
 436,
 448,
 449,
 476,
 490,
 502]

In [27]:
lineup = model.add_variables(players, name='lineup', vartype=so.binary)

In [30]:
lineup[11]

sasoptpy.Variable(name='lineup[11]', lb=0, ub=1, vartype='BIN')

In [34]:
bench = model.add_variables(players, name='bench',vartype=so.binary)

In [37]:
total_xp = so.expr_sum(lineup[p] * gk_data.loc[p, 'value_season'] for p in players) + 0.1 * (so.expr_sum(bench[p] * gk_data.loc[p, 'value_season'] for p in players))

In [38]:
total_xp

sasoptpy.Expression(exp = 21.0 * lineup[11] + 15.9 * lineup[24] + 2.3 * lineup[27] + 6.0 * lineup[60] + 10.4 * lineup[68] + 22.7 * lineup[78] + 1.5 * lineup[101] + 17.6 * lineup[108] + 17.9 * lineup[129] + 4.8 * lineup[139] + 21.4 * lineup[156] + 2.3 * lineup[180] + 17.5 * lineup[185] + 22.4 * lineup[206] + 1.6 * lineup[217] + 22.2 * lineup[237] + 17.8 * lineup[271] + 21.7 * lineup[294] + 17.0 * lineup[321] + 19.2 * lineup[337] + 2.0 * lineup[364] + 19.6 * lineup[382] + 17.8 * lineup[399] + lineup[404] + 3.3 * lineup[416] + 15.6 * lineup[436] + 15.2 * lineup[448] + 4.4 * lineup[449] + 17.2 * lineup[476] + 0.9 * lineup[490] + 21.6 * lineup[502] + 2.1 * bench[11] + 1.59 * bench[24] + 0.23 * bench[27] + 0.6 * bench[60] + 1.04 * bench[68] + 2.27 * bench[78] + 0.15 * bench[101] + 1.76 * bench[108] + 1.79 * bench[129] + 0.48 * bench[139] + 2.14 * bench[156] + 0.23 * bench[180] + 1.75 * bench[185] + 2.24 * bench[206] + 0.16 * bench[217] + 2.22 * bench[237] + 1.78 * bench[271] + 2.17 * bench[2

In [39]:
model.set_objective(total_xp, name='total_xp_obj', sense=so.maximize)

sasoptpy.Expression(exp = 21.0 * lineup[11] + 15.9 * lineup[24] + 2.3 * lineup[27] + 6.0 * lineup[60] + 10.4 * lineup[68] + 22.7 * lineup[78] + 1.5 * lineup[101] + 17.6 * lineup[108] + 17.9 * lineup[129] + 4.8 * lineup[139] + 21.4 * lineup[156] + 2.3 * lineup[180] + 17.5 * lineup[185] + 22.4 * lineup[206] + 1.6 * lineup[217] + 22.2 * lineup[237] + 17.8 * lineup[271] + 21.7 * lineup[294] + 17.0 * lineup[321] + 19.2 * lineup[337] + 2.0 * lineup[364] + 19.6 * lineup[382] + 17.8 * lineup[399] + lineup[404] + 3.3 * lineup[416] + 15.6 * lineup[436] + 15.2 * lineup[448] + 4.4 * lineup[449] + 17.2 * lineup[476] + 0.9 * lineup[490] + 21.6 * lineup[502] + 2.1 * bench[11] + 1.59 * bench[24] + 0.23 * bench[27] + 0.6 * bench[60] + 1.04 * bench[68] + 2.27 * bench[78] + 0.15 * bench[101] + 1.76 * bench[108] + 1.79 * bench[129] + 0.48 * bench[139] + 2.14 * bench[156] + 0.23 * bench[180] + 1.75 * bench[185] + 2.24 * bench[206] + 0.16 * bench[217] + 2.22 * bench[237] + 1.78 * bench[271] + 2.17 * bench[2

In [41]:
model.add_constraints((lineup[p]+ bench[p] <= 1 for p in players), name='lineup_or_bench');

In [42]:
model.add_constraint(so.expr_sum(lineup[p] for p in players) ==1, name = 'single_lineup');

In [45]:
model.add_constraint(so.expr_sum(bench[p] for p in players) ==1, name = 'single_bench');

In [49]:
budget = 9.2
model.add_constraint(so.expr_sum((lineup[p]+ bench[p])* gk_data.loc(p,'value') for p in players) <= budget, name='budget_con')

TypeError: __call__() takes from 1 to 2 positional arguments but 3 were given